In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [3]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower())
for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

104 documents
20 classes ['I am NOT fine', 'I am __ years old', 'I am fine', 'age', 'goodbye', 'greeting', 'hobbies', 'how_are_you', 'joke', 'living_place', 'options', 'thanks', 'what is my age', 'what is my fav color', 'what is my fav place', 'what is my living place', 'what is my name', 'what is your fav color', 'what is your fav place', 'what is your name']
106 unique lemmatized words ["'s", ')', ',', '.', ':', '__', '___', 'a', 'about', 'address', 'again', 'age', 'am', 'another', 'any', 'anyone', 'are', 'assist', 'awesome', 'bye', 'can', 'chatting', 'color', 'could', 'current', 'do', 'doing', 'excellent', 'favorite', 'fine', 'for', 'free', 'go', 'good', 'goodbye', 'great', 'guess', 'happy', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hobby', 'hola', 'how', 'hue', 'i', 'in', 'is', 'it', 'joke', 'know', 'later', 'laugh', 'like', 'live', 'location', 'love', 'make', 'me', 'most', 'my', 'name', 'next', 'nice', 'not', 'old', 'one', 'place', 'preferred', 'provide', 'remem

In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
# create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
train_x = []
train_y = []
for i in range(64):
    train_x.append(training[i][0])
    train_y.append(training[i][1])
# create train and test lists. X - patterns, Y - intents
print("Training data created")

Training data created


In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:73: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - accuracy: 0.0635 - loss: 3.0069    
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - accuracy: 0.0116 - loss: 2.9986   
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 622us/step - accuracy: 0.1073 - loss: 2.9293
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - accuracy: 0.1981 - loss: 2.8459
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - accuracy: 0.2651 - loss: 2.6831
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - accuracy: 0.2129 - loss: 2.6862
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 585us/step - accuracy: 0.2537 - loss: 2.5795
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - accuracy: 0.1987 - loss: 2.4746   
Epoch 9/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 480us/step - accuracy: 0.3096 - loss: 2.1394
Epoch 10/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 0.3034 - loss: 2.0418
Epoch 11/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - accuracy: 0.3449 - loss: 2.0194
Epoch 12/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 

model created
